#### Exécuter si vous n'avez pas déjà installer ces deux packages

In [ ]:
#!pip install gensim
#!pip install python-Levenshtein

#### Import des librairies

In [8]:
import numpy as np
import pandas as pd
import gensim
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#### Import des données

In [2]:
df = pd.read_json('quotes-2020.json.bz2', compression='bz2', nrows=100000, lines=True)

In [4]:
df.head(1)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E


#### 1. Doc2Vec

#### On va créer une liste de TaggedDocument. Chaque TaggedDocument contient un Document, et un Tag (identifiant unique).

Deux manières de créer les Tagged-Data, on voit que la deuxieme est mieux car elle permet de mieux nettoyer les quotations : 

In [7]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['quotation'])]

In [28]:
tagged_data[0]

TaggedDocument(words=['[', 'd', ']', 'espite', 'the', 'efforts', 'of', 'the', 'partners', 'to', 'create', 'a', 'non-political', 'award', 'that', 'unites', 'the', 'jewish', 'people', ',', 'some', 'have', 'incorrectly', 'interpreted', 'the', 'participation', 'of', 'the', 'office', 'of', 'the', 'prime', 'minister', 'in', 'the', 'genesis', 'prize', 'as', 'bringing', 'a', 'political', 'dimension', 'to', 'this', 'important', 'initiative', ','], tags=['0'])

In [52]:
vocab_quotes = df.quotation.apply(gensim.utils.simple_preprocess)
tagged_data_2 = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(vocab_quotes)]

In [53]:
tagged_data_2[0]

TaggedDocument(words=['espite', 'the', 'efforts', 'of', 'the', 'partners', 'to', 'create', 'non', 'political', 'award', 'that', 'unites', 'the', 'jewish', 'people', 'some', 'have', 'incorrectly', 'interpreted', 'the', 'participation', 'of', 'the', 'office', 'of', 'the', 'prime', 'minister', 'in', 'the', 'genesis', 'prize', 'as', 'bringing', 'political', 'dimension', 'to', 'this', 'important', 'initiative'], tags=[0])

In [67]:
# On définit le modèle, il y a beaucoup de paramètres à étudier pour optimiser
#model = gensim.models.Doc2Vec(vector_size=300, window=10, min_count=5, workers=4, epochs=20)
model = gensim.models.Doc2Vec(vector_size=100, dbow_words= 1, dm=0,  window=5, seed=1337, min_count=5, workers=4,alpha=0.025, min_alpha=0.025)

In [68]:
# On construit le vocab nécessaire au training 
model.build_vocab(tagged_data_2)

In [69]:
# On entraine le modèle
model.train(tagged_data_2, total_examples=model.corpus_count, epochs=10)

In [75]:
# On cherche les 10 documents les plus similaires à celui-ci. 

tokens = "The climate crisis has already been solved.".split()
new_vector = model.infer_vector(tokens)
sims = model.dv.most_similar([new_vector])

In [76]:
sims

[(572, 0.7996737360954285),
 (10474, 0.7693420052528381),
 (58040, 0.7656100392341614),
 (410, 0.7579727172851562),
 (99586, 0.7566854953765869),
 (52299, 0.7529284358024597),
 (86798, 0.7456504702568054),
 (47475, 0.7411602735519409),
 (16440, 0.7354569435119629),
 (23771, 0.7274510860443115)]

In [81]:
print(str(df.iloc[[23771]]['quotation']))

23771    in response to the climate changing,
Name: quotation, dtype: object


In [ ]:
# Bon ben apparement ca marche assez bien, comment faire mieux désormais ? 

#### 2. Word2Vec

In [16]:
vocab_quotes = df.quotation.apply(gensim.utils.simple_preprocess)

In [29]:
vocab_quotes[0]

['espite',
 'the',
 'efforts',
 'of',
 'the',
 'partners',
 'to',
 'create',
 'non',
 'political',
 'award',
 'that',
 'unites',
 'the',
 'jewish',
 'people',
 'some',
 'have',
 'incorrectly',
 'interpreted',
 'the',
 'participation',
 'of',
 'the',
 'office',
 'of',
 'the',
 'prime',
 'minister',
 'in',
 'the',
 'genesis',
 'prize',
 'as',
 'bringing',
 'political',
 'dimension',
 'to',
 'this',
 'important',
 'initiative']

In [20]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [21]:
model.build_vocab(vocab_quotes, progress_per=1000)

In [22]:
model.train(vocab_quotes, total_examples=model.corpus_count, epochs=model.epochs)

(8831377, 11833700)

In [27]:
model.wv.most_similar("climate")

[('economic', 0.7644410133361816),
 ('structural', 0.7232383489608765),
 ('environmental', 0.7198588252067566),
 ('science', 0.7084198594093323),
 ('urgency', 0.7079110145568848),
 ('policy', 0.705802321434021),
 ('risks', 0.7036910653114319),
 ('impacts', 0.7033190131187439),
 ('global', 0.7024703025817871),
 ('reforming', 0.6982715725898743)]

In [ ]:
# Ca marche aussi, mais y a des mots qui ne sont pas reliés QUE au changement climatique comme 'policy' et qui
# ne peuvent pas servir de clé pour déterminer à eux tout seul qu'une quote est sur le changement climatique.

### Conclusion :  Se concentrer sur la première approche Doc2Vec à mon avis, en mettant plus de données dans l'algorithme, chercher à trouver les paramètres optimaux de train, améliorer le pré-processing en dégageant les mots tronqués, et en réajustant continuellement pour avoir la meilleure accuracy possible. 

#### Trouver comment importer toute la donnée en même temps ? Ou une maniere d'utiliser toute la donnée pour entrainer le modèle en tout cas. 

#### Aussi pourquoi quand on print kes